<a href="https://colab.research.google.com/github/MCanela-1954/Data/blob/main/%5BRAG_9%5D_QA_on_the_IESE_EMBA_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [RAG-9] Q/A on the IESE EMBA Program

## Setup

In [ ]:
!pip install lmstudio

In [ ]:
!pip install openai

In [ ]:
import lmstudio as lms, numpy as np, pandas as pd
from openai import OpenAI

## Model selection

In [ ]:
embed_model = 'text-embedding-granite-embedding-278m-multilingual'

In [ ]:
chat_model = 'ibm-granite/granite-3.3-8b-instruct-GGUF'
# chat_model = 'qwen3-14b'
# chat_model = 'mistral-small-3.1-24b-instruct-2503'
# chat_model = 'gemma-3-12b-it'

## LM Studio client

In [ ]:
client = OpenAI(base_url = 'http://localhost:1234/v1', api_key = 'lm-studio')

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/IAlegre-MCanela/Summer_Course/main/Session%209/faq.json')

In [ ]:
questions = df['question'].tolist()

In [ ]:
N = len(df)
N

In [ ]:
response = client.embeddings.create(model = embed_model, input = questions)

In [ ]:
embeds = [response.data[i].embedding for i in range(N)]
embeds = np.array(embeds)
embeds.shape

## Encoding the query

In [ ]:
query = 'Are courses taught in Spanish?'

In [ ]:
response = client.embeddings.create(model = embed_model, input = [query])

In [ ]:
query_embed = response.data[0].embedding
query_embed = np.array(query_embed)
query_embed.shape

## Search for the closest questions

In [ ]:
df['similarity'] = np.dot(embeds, query_embed.T)
df.head()

In [ ]:
top5 = df.sort_values(by='similarity', ascending=False).head(5)
top5

## System instructions

In [ ]:
instruction = '''
You are a chatbot providing an answer to a user question about MBA programs at IESE Business School.
You will be shown the user question.
Make your answer shorter than 50 words.
Use only the information provided.
If the information provided doesn't mention the subject of the query, limit your response to the following:
<response>
Unfortunately, I don't have information about this subject. Please, directly contact with the school.
</response>
'''

In [ ]:
info = 'Information:\n' + top5['answer'].sum()
print(info)

## Submitting the query (with potential reasoning)

In [ ]:
query_response = client.chat.completions.create(
    messages = [{'role': 'system', 'content': instruction + info}, {'role': 'user', 'content': query}],
    model = chat_model)
print(query_response.choices[0].message.content)